In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.distance import geodesic
import matplotlib.pyplot as plt
import seaborn as sns
import os
pd.set_option('display.max_columns', None)

In [ ]:
# main csv file
df = pd.read_csv('housing.csv')
# auxiliary csv files
cal_cities = pd.read_csv('cal_cities_lat_long.csv')
cal_pops_cities = pd.read_csv('cal_populations_city.csv')
cal_pops_counties = pd.read_csv('cal_populations_county.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[['ocean_proximity']].value_counts()

In [ ]:
# train test split
# 80% train, 20% test
# random_state=42 for reproducibility
# stratify=df['ocean_proximity'] to ensure that the train and test sets have the same proportion of each category as the full dataset
# we also want to make sure that we dont include the median_house_value in the train set
from sklearn.model_selection import train_test_split
X = df.drop('median_house_value', axis=1)
y = df['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df.loc[df['ocean_proximity'] == '<1H OCEAN', 'ocean_proximity'] = 'WITHIN HOUR TO OCEAN'

In [ ]:
df[['ocean_proximity']].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df[['total_bedrooms']].describe()

In [ ]:
df.head()

In [ ]:
# looking at the histogram, we can see that the data is skewed to the right
df[['total_bedrooms']].hist(bins=50)

In [ ]:
df[['total_bedrooms']].boxplot()

In [ ]:
# let's fill the missing values with the median
df['total_bedrooms'].fillna(df['total_bedrooms'].median(), inplace=True)   

In [ ]:
df.isnull().sum() # no more missing values

In [ ]:
# now that we have all our data, let's investigate even further
# we'll start by looking at the correlation between the variables
number_cols = df.select_dtypes(include=['int64', 'float64'])
number_cols.drop(['longitude', 'latitude'], axis=1, inplace=True)
number_cols.corr()

In [ ]:
# let's create a correlation matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.heatmap(number_cols.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# let's look at the scatter plots of the variables that are highly correlated
sns.pairplot(number_cols[['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']])
plt.show()

In [ ]:
# let's do some feature engineering
# we'll start by creating a new variable that will be the ratio of total_rooms to households
df['rooms_per_household'] = df['total_rooms'] / df['households']

# let's create another variable that will be the ratio of total_bedrooms to total_rooms
df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']

# let's create another variable that will be the ratio of population to households
df['population_per_household'] = df['population'] / df['households']

# let's look at the correlation matrix again
plt.figure(figsize=(12, 8))
number_cols = df.select_dtypes(include=['int64', 'float64'])
number_cols.drop(['longitude', 'latitude'], axis=1, inplace=True)
sns.heatmap(number_cols.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Is total_rooms still necessary?
df.drop(['total_rooms'], axis=1, inplace=True)

In [ ]:
# Is total_bedrooms still necessary?
df.drop(['total_bedrooms'], axis=1, inplace=True)

In [ ]:
# Is population still necessary?
# df.drop(['population'], axis=1, inplace=True)
# I think it's still necessary

In [ ]:
# Is households still necessary?
df.drop(['households'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# let's look at the correlation matrix again
plt.figure(figsize=(12, 8))
number_cols = df.select_dtypes(include=['int64', 'float64'])
number_cols.drop(['longitude', 'latitude'], axis=1, inplace=True)
sns.heatmap(number_cols.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# let's make geographical plots
# let's start by looking at the geographical distribution of the median house value

# Load the California shapefile
file_path = os.path.join('CA_Counties', 'CA_Counties_TIGER2016.shp')
cali_shp = gpd.read_file(file_path)

# Create the GeoDataFrame for your points
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']), crs=cali_shp.crs)

# Set the CRS of the points GeoDataFrame to match the CRS of the California GeoDataFrame
gdf.set_crs(cali_shp.crs, inplace=True)

In [ ]:
# Load the California shapefile
# cali = gpd.read_file('CA_Counties\CA_Counties_TIGER2016.shp')

file_path = os.path.join('CA_Counties', 'CA_Counties_TIGER2016.shp')
cali_shp = gpd.read_file(file_path)
cali_shp = cali_shp.to_crs("EPSG:4326")

# Create the GeoDataFrame for your points
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']), crs=cali_shp.crs)
gdf = gdf.to_crs("EPSG:4326")

In [ ]:
# Plotting
ax = cali_shp.plot(color='grey',figsize=(10, 10), alpha=0.4, edgecolor='blue')
# gdf.plot(ax=ax, color='red')
gdf.plot(ax=ax, kind='scatter', x='longitude', y='latitude', alpha=0.4, s=gdf['population']/500, label='population', figsize=(10,10), 
                        c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True)


In [ ]:
ax = cali_shp.plot(color='grey',figsize=(10, 10), alpha=0.4, edgecolor='blue')
gdf.plot(ax=ax, kind='scatter', x='longitude', y='latitude', alpha=0.4, s=gdf['median_income']/1, label='population', figsize=(10,10), 
                        c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True)

In [ ]:
# let's look at the geographical distribution of the median income
ax = cali_shp.plot(color='grey',figsize=(10, 10), alpha=0.4, edgecolor='blue')
gdf.plot(ax=ax, kind='scatter', x='longitude', y='latitude', alpha=0.4, s=gdf['median_income']/1, label='population', figsize=(10,10))

In [ ]:
cal_cities.head()

In [ ]:
cal_pops_cities.head()

In [ ]:
cal_pops_counties.head()

In [ ]:
from collections import namedtuple

CityTuple = namedtuple('City', ['Name', 'Latitude', 'Longitude','pop_april_1980', 'pop_april_1990', 'pop_april_2000', 'pop_april_2010'])

city_map = dict()
for index, row in cal_cities.iterrows():
    city_map[row['Name']] = CityTuple(row['Name'], row['Latitude'], row['Longitude'], 0, 0, 0, 0)

In [ ]:
for index, row in cal_pops_cities.iterrows():
    if row['City'] in city_map:
        tuple_ = city_map[row['City']]
        tuple_ = tuple_._replace(pop_april_1980=row['pop_april_1980'], pop_april_1990=row['pop_april_1990'], pop_april_2000=row['pop_april_2000'], pop_april_2010=row['pop_april_2010'])
        city_map[row['City']] = tuple_

In [ ]:
# We're going to create a geodataframe for the cities
from shapely.geometry import Point

tuple_list = [tuple_ for tuple_ in city_map.values()]

# Create the GeoDataFrame for your points from the tuple_list
gdf_cities = gpd.GeoDataFrame(tuple_list, geometry=gpd.points_from_xy([tuple_[2] for tuple_ in tuple_list], [tuple_[1] for tuple_ in tuple_list]), crs=cali_shp.crs)
gdf_cities = gdf_cities.to_crs("EPSG:4326")


In [ ]:
# I do not want to map more geographical features like counties.
# I think the distance from large cities is enough

# let's create a new column for the distance from large cities
# first we'll plot large cities and their population

gdf_cities[['pop_april_1980','pop_april_1990','pop_april_2000','pop_april_2010']] = gdf_cities[['pop_april_1980','pop_april_1990','pop_april_2000','pop_april_2010']].astype(float)
gdf_cities['Large_City'] = gdf_cities['pop_april_2010'] > 250000
large_cities = gdf_cities[gdf_cities['Large_City'] == True]
large_cities.head()

In [ ]:
ax = cali_shp.plot(color='grey',figsize=(10, 10), alpha=0.4, edgecolor='blue')
large_cities.plot(ax=ax, kind='scatter', x='Longitude', y='Latitude', alpha=0.4, s=large_cities['pop_april_2010'] / 10000, label='pop_april_2010', figsize=(10,10), c='pop_april_2010', cmap=plt.get_cmap('jet'), colorbar=True)

In [ ]:
import matplotlib.pyplot as plt

# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(20, 20))

# Flatten the axes array for easier iteration
axs = axs.flatten()

# List of years to plot
years = ['pop_april_1980','pop_april_1990','pop_april_2000','pop_april_2010']

# Iterate over years and axes together
for ax, year in zip(axs, years):
    cali_shp.plot(color='grey', ax=ax, alpha=0.4, edgecolor='blue')
    large_cities.plot(kind='scatter', x='Longitude', y='Latitude', alpha=0.4, 
                      s=large_cities[year] / 10000, label=year, ax=ax, 
                      c=large_cities[year], cmap=plt.get_cmap('jet'), colorbar=True)
    ax.legend()

# Adjust the layout
plt.tight_layout()
plt.show()


In [ ]:
# Let's get the distance from large cities
from collections import OrderedDict


large_cities_lat_lon = large_cities[['Name','Latitude', 'Longitude']]
large_cities_dictionaries = large_cities_lat_lon.to_dict('records', into=OrderedDict)

In [ ]:
gdf.head()

In [ ]:
# Need two tuples of lat and lon for city and location

for large_city in large_cities_dictionaries:
    large_city_base_name = large_city['Name']
    large_city_base_name = large_city_base_name.replace(' ', '_').lower()
    large_city_enriched_name = large_city_base_name + '_km_distance'
    gdf[large_city_enriched_name] = gdf.apply(lambda row: geodesic((row['latitude'], row['longitude']), (large_city['Latitude'], large_city['Longitude'])).kilometers, axis=1)


In [ ]:
gdf.head()

In [ ]:
# Get all column names that end with '_km_distance'
# parsed_string = s.split('_km_distance')[0].replace('_', ' ')
# distance_cols = [(col.split('_km_distance')[0].replace('_', ' '), col) for col in gdf.columns if col.endswith('_km_distance')]
distance_cols = [col for col in gdf.columns if col.endswith('_km_distance')]
distance_cols

# Select only these columns
distance_data = gdf[distance_cols]
distance_data.head()    

# # Find the minimum distance for each row
gdf['min_distance'] = distance_data.min(axis=1)
gdf['max_distance'] = distance_data.max(axis=1)

# Find the column name where column equals the min for each row
# column name must be unique in terms of distance_km since we're going to need to regex that and drop those columns to save space
gdf['min_distance_km_col'] = distance_data.idxmin(axis=1)
gdf['max_distance_km_col'] = distance_data.idxmax(axis=1)

gdf.drop(columns=distance_cols, inplace=True)


In [ ]:
gdf.head()

In [ ]:
gdf.tail()

In [ ]:
gdf['min_distance_city_name'] = gdf['min_distance_km_col'].apply(lambda x: x.split('_km_distance')[0].replace('_', ' '))
gdf.drop(columns=['min_distance_km_col', 'max_distance_km_col', 'max_distance'], inplace=True)

In [ ]:
gdf.head()

In [ ]:
gdf.drop(columns=['latitude', 'longitude', 'max_distance_city_name'], inplace=True)

In [ ]:
gdf.head()

In [ ]:
# Create a single subplot
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# cali_shp.plot(color='grey', ax=ax, alpha=0.4, edgecolor='blue')

# Plot the geometry column of gdf on the ax
gdf.plot(column='min_distance_city_name', ax=ax, alpha=0.4, 
         edgecolor='white', legend=True, cmap='jet')

# Adjust the layout
# plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ax = cali_shp.plot(color='grey',figsize=(10, 10), alpha=0.4, edgecolor='blue')

# Plot the geometry column of gdf on the ax
gdf.plot(column='min_distance_city_name', ax=ax, alpha=0.4, 
         edgecolor='white', legend=True, cmap='jet')

# Show the plot
ax.axis('off')

plt.show()

In [ ]:
gdf.head()

In [ ]:
large_cities.head()

In [ ]:
large_cities.shape

In [ ]:
# We now need to get the population of the city that is closest to each row
# We can do this by merging the gdf with the large_cities dataframe
# We'll need to rename the columns in large_cities to match the column names in gdf
# We'll also need to drop the geometry column from large_cities
# We'll also need to drop the geometry column from gdf

# Rename the columns in large_cities
# large_cities = large_cities.rename(columns={'min_distance_city_name': 'city_name'})
# large_cities['min_distance_city_name'] = large_cities['min_distance_city_name'].apply(lambda x: x.lower())
# large_cities.reset_index(inplace=True)
# large_cities.drop(columns=['index'], inplace=True)

large_cities = large_cities.rename(columns={'Name': 'city_name_pops'})
large_cities['city_name_pops'] = large_cities['city_name_pops'].apply(lambda x: x.lower())
large_cities['city_name_pops'] = large_cities['city_name_pops'].apply(lambda x: x.replace(' ', '_'))
large_cities.head(len(large_cities))

In [ ]:
gdf['min_distance_city_name'] = gdf['min_distance_city_name'].apply(lambda x: x.replace(' ', '_'))
gdf['min_distance_city_name'].value_counts()

In [ ]:
merged_gdf = gdf.merge(large_cities, how='left', left_on='min_distance_city_name', right_on='city_name_pops')

In [ ]:
merged_gdf.head()

In [ ]:
merged_gdf.drop(columns=['city_name_pops', 'Latitude', 'Longitude', 'geometry_y', 'Large_City'], inplace=True)

In [ ]:
merged_gdf.rename(columns={'geometry_x': 'geometry'}, inplace=True)
merged_gdf.head()

In [ ]:
merged_gdf.drop(columns=['geometry'], inplace=True)
merged_gdf.head()

In [ ]:
# We now have to feature engineer ocean_proximity to be a categorical variable, and then one-hot encode it
# We'll also need to drop the ocean_proximity column
# We'll also need to remove outliers from columns as well
number_cols = merged_gdf.select_dtypes(include=np.number)
# let's look at the scatter plots of the variables that are highly correlated
sns.pairplot(number_cols)
plt.show()

In [ ]:
# Using pandas
# merged_gdf.hist(bins=30, layout=(merged_gdf.shape[1], 1), figsize=(6, 6*merged_gdf.shape[1]))
# plt.tight_layout()
# plt.show()

# Using seaborn
for column in merged_gdf.select_dtypes(include=np.number):
    plt.figure()
    sns.histplot(merged_gdf[column], kde=True)
plt.show()

In [ ]:
# based off of these histograms, we need to normalize the following columns:
# log for population of each city
# pop_april_1980 pop_april_1990 pop_april_2000 pop_april_2010

merged_gdf[['pop_april_1980', 'pop_april_1990', 'pop_april_2000', 'pop_april_2010']] = merged_gdf[['pop_april_1980', 'pop_april_1990', 'pop_april_2000', 'pop_april_2010']].apply(np.log)

In [ ]:
merged_gdf.head()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [ ]:
list(merged_gdf.columns)

In [ ]:
num_processor = Pipeline([("std_scaler", StandardScaler())])

cat_processor = Pipeline(
    [("one_hot_encoder", OneHotEncoder(sparse=False, handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    [
        (
            "num",
            num_processor,
            [
                "housing_median_age",
                "population",
                "median_income",
                "median_house_value",
                "rooms_per_household",
                "bedrooms_per_room",
                "population_per_household",
                "min_distance",
                "pop_april_1980",
                "pop_april_1990",
                "pop_april_2000",
                "pop_april_2010",
            ],
        ),
        ("cat", cat_processor, ["min_distance_city_name", "ocean_proximity"]),
    ]
)